# Biol 359A | Statistical Tests: Linear Regression
### Spring 2025, Week 3
Objectives:
- Interact with real data
- Learn how to fit lines to data

In [ ]:
# Import necessary libraries
from ipywidgets import interact, IntSlider, FloatSlider, Layout, Dropdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, f, f_oneway
from scipy import stats
import seaborn as sns
import statsmodels.stats.multicomp as mc
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import sklearn as sk
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [ ]:
! rm -r week3_linearRegression/
! git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr25/week3_linearRegression.git
! cp -r week3_linearRegression/* .
! ls

For today's lesson we will be working on real breast cancer data from the[ Wisconsin Diagnostic Breast Cancer Database (WDBC)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic).

Here is a summary of the data from the data source:
```
	Features are computed from a digitized image of a fine needle
	aspirate (FNA) of a breast mass.  They describe
	characteristics of the cell nuclei present in the image.
	A few of the images can be found at
	http://www.cs.wisc.edu/~street/images/

	Separating plane described above was obtained using
	Multisurface Method-Tree (MSM-T) [K. P. Bennett, "Decision Tree
	Construction Via Linear Programming." Proceedings of the 4th
	Midwest Artificial Intelligence and Cognitive Science Society,
	pp. 97-101, 1992], a classification method which uses linear
	programming to construct a decision tree.  Relevant features
	were selected using an exhaustive search in the space of 1-4
	features and 1-3 separating planes.

	The actual linear program used to obtain the separating plane
	in the 3-dimensional space is that described in:
	[K. P. Bennett and O. L. Mangasarian: "Robust Linear
	Programming Discrimination of Two Linearly Inseparable Sets",
	Optimization Methods and Software 1, 1992, 23-34].

	This database is also available through the UW CS ftp server:
	ftp ftp.cs.wisc.edu
	cd math-prog/cpo-dataset/machine-learn/WDBC/
    
    Source:
    W.N. Street, W.H. Wolberg and O.L. Mangasarian
	Nuclear feature extraction for breast tumor diagnosis.
	IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science
	and Technology, volume 1905, pages 861-870, San Jose, CA, 1993.
```

What do all the column names mean?

- ID number
- Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry
- fractal dimension ("coastline approximation" - 1) - a measure of "complexity" of a 2D image.


Cateogory Distribution: 357 benign, 212 malignant

In [ ]:
import clean_data

cancer_dataset = clean_data.generate_clean_dataframe()
cancer_dataset = cancer_dataset.reset_index()
cancer_dataset.drop("ID", axis=1, inplace=True)
cancer_dataset

#### In order for us to use diagnosis as a variable, we need to turn it from categorical (M, B) into numerical values. We will set "M" to 1, and "B" to 0. 

In [ ]:
cancer_dataset['diagnosis'].replace(["M","B"], [1,0], inplace=True)
cancer_dataset

## Fitting lines to data

### Simple linear regression


Fitting lines to data, or linear regression, learns the best linear relationship between two feature (X) and outcome (Y) by minimizing the sum of squared errors (SSE) between predicted and actual values. This fitted line can then be used to predict the value of one variable given the other.

Additionally, it helps quantify the strength and direction of the relationship (via the slope), and can provide insights into how changes in X are associated with changes in Y.

\begin{align*}
S_{xy} &= \sum_{i=1}^{n} (y_i - \bar{y})(x_i - \bar{x}) \\
S_{xx} &= \sum_{i=1}^{n} (x_i - \bar{x})^2 \\
\\
\text{Coefficients are:} \\
\hat{\beta}_1 &= \frac{S_{xy}}{S_{xx}} \\
\hat{\beta}_0 &= \mathbb{E}[Y] - \hat{\beta}_1 \mathbb{E}[X]
\end{align*}



In [ ]:
def calculate_beta_coefficients(x, y):
    """
    Calculate beta coefficients using least squares method.

    Parameters:
    x (array-like): Independent variable values
    y (array-like): Dependent variable values

    Returns:
    tuple: (beta_0, beta_1, S_xy, S_xx)
    """
    # Convert to numpy arrays if they aren't already
    x = np.array(x)
    y = np.array(y)

    # Calculate means
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    S_xy = np.sum((y - y_mean) * (x - x_mean))
    S_xx = np.sum((x - x_mean) ** 2)
    S_yy = np.sum((y - y_mean) ** 2)

    beta_1 = S_xy / S_xx
    beta_0 = y_mean - beta_1 * x_mean

    return beta_0, beta_1, S_xy, S_xx, S_yy

@widgets.interact(x_feature=list(cancer_dataset), y_feature=list(cancer_dataset))
def make_interactive_regression(x_feature="mean_radius", y_feature="mean_perimeter"):
    """
    Interactive function to run regression analysis on selected features using decorator syntax.
    
    Parameters:
    x_feature (str): Name of the independent variable column
    y_feature (str): Name of the dependent variable column
    """
    # Extract the variables
    x = cancer_dataset[x_feature]
    y = cancer_dataset[y_feature]
    
    # Calculate beta coefficients
    beta_0, beta_1, S_xy, S_xx, S_yy = calculate_beta_coefficients(x, y)
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Plot scatter points
    plt.scatter(x, y, color='blue', alpha=0.6, label='Data points')
    
    # Plot regression line
    x_range = np.linspace(min(x), max(x), 100)
    y_pred = beta_0 + beta_1 * x_range
    plt.plot(x_range, y_pred, color='red', linewidth=2, 
             label=f'Fitted line: y = {beta_0:.4f} + {beta_1:.4f}x')
    
    # Add labels and title
    plt.xlabel(x_feature)
    plt.ylabel(y_feature)
    plt.title(f'Linear Regression: {y_feature} vs {x_feature}')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Display equation and statistics on the plot
    equation = f"{y_feature} = {beta_0:.4f} + {beta_1:.4f} × {x_feature}"
    stats = f"S_xy: {S_xy:.4f}, S_xx: {S_xx:.4f}"
    
    plt.annotate(equation, xy=(0.05, 0.95), xycoords='axes fraction',
                 fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    plt.annotate(stats, xy=(0.05, 0.89), xycoords='axes fraction',
                 fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print the results
    print(f"S_xy: {S_xy:.4f}")
    print(f"S_xx: {S_xx:.4f}")
    print(f"Correlation coefficient: {S_xy/np.sqrt(S_xx*S_yy)}")
    print(f"Beta 1 (slope): {beta_1:.4f}")
    print(f"Beta 0 (intercept): {beta_0:.4f}")
    print(f"Regression equation: {y_feature} = {beta_0:.4f} + {beta_1:.4f} × {x_feature}")

### Metrics for linear regression

We are going to introduce the concept of the __Coefficient of Determination__: $R^2$. 

$$ R^2 = \frac{SSR}{SST} = 1 - \frac{SSE}{SST}$$

Where 
- SSE = error of sum of squares
- SSR = regression sum of squares
- SST = total sum of squares

This value conceptually represents the "amount of variance in y is explained by x" and can be thought of as how well the model fits the training data. This is a seperate concept to the Pearson Correlation Coefficient, $\rho$, which in the cases of a __strictly linear regression__, $R^2 = \rho^2$. 

You can also calculate the adjusted $R^2$ which accounts for the model size and parameters. This way, you can compare the adjusted $R^2$ of simple models to the adjusted $R^2$ of complex models, which you can't necessarily do if you just use  $R^2$. 

$$R^2_{adj} = 1 - \frac{\frac{SSE}{n-k-1}}{\frac{SST}{n - 1}}$$

Where 
- n = number of data points
- k number of independent variables

Generally speaking, $R^2$ is used to characterize the fit of a model, where as $\rho$ is used to characterize the relationship between two variables. 

We will perform a simple linear regression between the response variable, y, and the feature, x. 

In [ ]:
def calculate_r_squared(x, y, beta_0, beta_1):
    """
    Calculate the coefficient of determination (R²).
    
    Parameters:
    x (array-like): Independent variable values
    y (array-like): Dependent variable values
    beta_0 (float): Intercept coefficient
    beta_1 (float): Slope coefficient
    
    Returns:
    float: R² value
    """
    # Convert to numpy arrays if they aren't already
    x = np.array(x)
    y = np.array(y)
    
    # Calculate predicted values
    y_pred = beta_0 + beta_1 * x
    
    # Calculate total sum of squares (TSS)
    y_mean = np.mean(y)
    TSS = np.sum((y - y_mean) ** 2)
    
    # Calculate residual sum of squares (RSS)
    RSS = np.sum((y - y_pred) ** 2)
    
    # Calculate R-squared
    r_squared = 1 - (RSS / TSS)
    
    return r_squared

@widgets.interact(x_feature=list(cancer_dataset), y_feature=list(cancer_dataset))
def make_interactive_regression(x_feature="mean_radius", y_feature="mean_perimeter"):
    """
    Interactive function to run regression analysis on selected features using decorator syntax.
    
    Parameters:
    x_feature (str): Name of the independent variable column
    y_feature (str): Name of the dependent variable column
    """
    # Extract the variables
    x = cancer_dataset[x_feature]
    y = cancer_dataset[y_feature]
    
    # Calculate beta coefficients
    beta_0, beta_1, S_xy, S_xx, S_yy = calculate_beta_coefficients(x, y)
    
    # Calculate R-squared
    r_squared = calculate_r_squared(x, y, beta_0, beta_1)
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Plot scatter points
    plt.scatter(x, y, color='blue', alpha=0.6, label='Data points')
    
    # Plot regression line
    x_range = np.linspace(min(x), max(x), 100)
    y_pred = beta_0 + beta_1 * x_range
    plt.plot(x_range, y_pred, color='red', linewidth=2, 
             label=f'Fitted line: y = {beta_0:.4f} + {beta_1:.4f}x')
    
    # Add labels and title
    plt.xlabel(x_feature)
    plt.ylabel(y_feature)
    plt.title(f'Linear Regression: {y_feature} vs {x_feature}')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Display equation and statistics on the plot
    equation = f"{y_feature} = {beta_0:.4f} + {beta_1:.4f} × {x_feature}"
    stats = f"R² = {r_squared:.4f}, S_xy: {S_xy:.4f}, S_xx: {S_xx:.4f}"
    
    plt.annotate(equation, xy=(0.05, 0.95), xycoords='axes fraction',
                 fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    plt.annotate(stats, xy=(0.05, 0.89), xycoords='axes fraction',
                 fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print the results
    print(f"S_xy: {S_xy:.4f}")
    print(f"S_xx: {S_xx:.4f}")
    print(f"Correlation coefficient: {S_xy/np.sqrt(S_xx*S_yy)}")
    print(f"Beta 1 (slope): {beta_1:.4f}")
    print(f"Beta 0 (intercept): {beta_0:.4f}")
    print(f"R-squared: {r_squared:.4f}")
    print(f"Regression equation: {y_feature} = {beta_0:.4f} + {beta_1:.4f} × {x_feature}")

### Assignment 3!
[template](https://colab.research.google.com/drive/1dhnxVqRQli5bQPGIQl3RnzCer0X94vGS?usp=sharing)

### Multiple Linear Regression 

We have other variables that we can use to explore the relationships in this dataset. We can use multiple linear regression to include more variables, where we are drawing a multi-dimensional shape, rather than simply a line, as our model. The plot below is currently the same as the one we looked at above with mean_area and mean_radius. Try adding some other variables into your regression by checking the boxes.

In [ ]:
TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (12,12)
COLORS= ["#008080","#CA562C"]

sns.set_context("notebook")
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
sns.set(font_scale=1, rc={'figure.figsize':FIG_SIZE}) 

In [ ]:
def linear_regression(df, feature_cols, response_col, standardized = False):
    """
    Use linear_model to run a linear regression using sklearn
    
    """
    X = df[feature_cols]
    y = df[response_col]
    if standardized:
        X = StandardScaler().fit_transform(X)
        y = StandardScaler().fit_transform(y.values.reshape(-1, 1))
    regression = linear_model.LinearRegression() 
    regression.fit(X,y)
    try:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_))
    except TypeError:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_[0]))
    print('Regression Coefficients: (coefficients for linear combinations)')
    for feature, coef in zip(feature_cols, regression.coef_.flatten()):
        print(f'{feature} ~ {coef:.2f}')
    return regression.predict(X), regression.score(X,y)
    
def parity_plot(true, pred, r_squared=None, title='', hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(x=true, y=pred, hue=hue)
    else: 
        sns.scatterplot(x=true, y=pred)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    ax.set_box_aspect(1)
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()


def run_regression(feature_cols = 
                   ['mean_radius', 
                    'mean_texture', 
                    'mean_perimeter', 
                    'mean_smoothness', 
                    'mean_compactness', 
                    'mean_concavity', 
                    'mean_concave_points', 
                    'mean_symmetry', 
                    'mean_fractal_dimension'], 
                     response_col='mean_area',
                     standardized=False,
                     parity=True):
    y_pred, r_squared = linear_regression(cancer_dataset, feature_cols, response_col, standardized = standardized)
    if parity: parity_plot(cancer_dataset[response_col], y_pred.flatten(), r_squared, hue=cancer_dataset["diagnosis"])


@widgets.interact(response=list(cancer_dataset))   
def regression_wrapper(response="mean_radius",
                       diagnosis=False,
                       radius=False, 
                       texture=False, 
                       perimeter=False, 
                       area=True,
                       smoothness=False, 
                       compactness=False, 
                       concavity=False, 
                       concave_points=False, 
                       symmetry=False, 
                       fractal_dimension=False):
    features = []
    if diagnosis: features.append("diagnosis")
    if radius: features.append("mean_radius")
    if texture: features.append("mean_texture")
    if perimeter: features.append("mean_perimeter")
    if area: features.append("mean_area")
    if smoothness: features.append("mean_smoothness")
    if compactness: features.append("mean_compactness")    
    if concavity: features.append("mean_concavity")
    if concave_points: features.append("mean_concave_points")
    if symmetry: features.append("mean_symmetry")
    if fractal_dimension: features.append("mean_fractal_dimension")
        

    run_regression(feature_cols=features, response_col = response)